In [1]:
import datasets
import numpy as np

data = datasets.load_from_disk('all_characters')

print(len(data))

d:\miniconda\envs\conda-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


274


In [2]:
import random

def add_random_noise(image):
    noise_type = random.choice(["gaussian", "salt_pepper", "speckle"])
    
    if noise_type == "gaussian":
        mean = 0
        stddev = 0.1  # Adjust noise level
        gaussian_noise = np.random.normal(mean, stddev, image.shape)
        noisy_image = np.clip(image + gaussian_noise, 0, 1)
    
    elif noise_type == "salt_pepper":
        amount = 0.05  # Adjust noise density
        noisy_image = np.copy(image)
        num_salt = np.ceil(amount * image.size * 0.5).astype(int)
        num_pepper = np.ceil(amount * image.size * 0.5).astype(int)
        
        # Add salt
        coords = [np.random.randint(0, i, num_salt) for i in image.shape]
        noisy_image[coords[0], coords[1]] = 1
        
        # Add pepper
        coords = [np.random.randint(0, i, num_pepper) for i in image.shape]
        noisy_image[coords[0], coords[1]] = 0
    
    elif noise_type == "speckle":
        noise = np.random.randn(*image.shape) * 0.1  # Adjust noise level
        noisy_image = np.clip(image + image * noise, 0, 1)
    
    return noisy_image

In [3]:
import skimage.transform as sk

new_shape = (40, 40)
def scale_input(data):
    input, output = data['image'], data['id']
    input = np.array(input, dtype=np.float64)
    input = add_random_noise(input)
    input = sk.resize(np.array(input), new_shape, mode='reflect', anti_aliasing=True)
    return {'input': input, 'label': output}

train = []
test = []

train_loop = 100
test_loop = 10

for i in range(len(data)):
    for _ in range(train_loop):
        train.append(scale_input(data[i]))
    for _ in range(test_loop):
        test.append(scale_input(data[i]))

In [4]:
def image_print(tab):
    for r in tab:
        for c in r:
            if c < 0.2: print(' ', end='')
            elif c < 0.4: print('.', end='')
            elif c < 0.6: print('o', end='')
            elif c < 0.6: print('O', end='')
            else: print('@', end='')
        print()

image_print(train[0]['input'])

..                               .o@@@..
                oo      oo   .o@@@@@.   
                       .@o .@@@@.       
                     .@@@@@@.           
                 .@@@@@@@o.             
              .o@@@@@@@o            ..  
            .@@@@@@@@.                  
          o@@@@@@@o.    ..            ..
         .@@@@@@@@.   oo                
       .@@@@@@@@.                       
      .o@@@@@@.                         
    .o@@@@@@@@.         oo              
   .@@@@@@@@.               .o..        
  .@@@@@@@@@.                           
..@@@@@@@@@@.                           
..@@@@@@@@.         oo            oo    
@@@@@@@@@@.                 oo          
@@@@@@@@@@.                             
@@@@@@@@@@.                             
@@@@@@@@@@.       ..      ..            
@@@@@@@@@@.                             
@@@@@@@@@@.                             
@@@@@@@@@@.                             
..@@@@@@@@.                             
..@@@@@@@@.     

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters
input_channels = 1  # 1 for grayscale, 3 for RGB
image_size = 40
hidden_size = 512
output_size = 274
learning_rate = 0.001
batch_size = 64
num_epochs = 2

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, input_channels, hidden_size, output_size):
        super(CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1)  # (40x40) -> (40x40)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # (40x40) -> (20x20)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # (20x20) -> (20x20)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # (20x20) -> (10x10)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)  # (20x20) -> (20x20)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # (10x10) -> (5x5)

        # Fully connected layers
        self.fc1 = nn.Linear(32*10*10, hidden_size)  # Flattened size
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))  # Conv1 -> ReLU -> Pool
        x = self.pool2(torch.relu(self.conv2(x)))  # Conv2 -> ReLU -> Pool
        x = self.pool3(torch.relu(self.conv3(x)))  # Conv3 -> ReLU -> Pool
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))  # Fully connected
        x = self.fc2(x)  # Output layer
        return x

# Initialize model
model = CNN(input_channels, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Prepare dataset (reshape to 40x40 for CNN)
X_train = torch.stack([torch.tensor(data['input'], dtype=torch.float32).reshape(1, image_size, image_size) for data in train]).to(device)
y_train = torch.tensor([data['label'] for data in train], dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

X_test = torch.stack([torch.tensor(data['input'], dtype=torch.float32).reshape(1, image_size, image_size) for data in test]).to(device)
y_test = torch.tensor([data['label'] for data in test], dtype=torch.long).to(device)

# Ensure all tensors are moved to the correct device
X_test = X_test.to(device)
y_test = y_test.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = len(test)
        sum_loss = 0

        for i in range(total):
            output = model(X_test[i].unsqueeze(0).to(device))  # Add batch dimension and move to device
            predicted = torch.argmax(output, dim=1).item()

            if predicted == y_test[i].item():
                correct += 1

            sum_loss += criterion(output, y_test[i].unsqueeze(0).to(device)).item()

        avg_loss = sum_loss / total
        accuracy = correct / total

        print(f"Epoch {epoch+1}: Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

cuda
Epoch [1/2], Step [1/429], Loss: 5.6105
Epoch [1/2], Step [1/429], Loss: 5.6105
Epoch [1/2], Step [101/429], Loss: 0.2307
Epoch [1/2], Step [101/429], Loss: 0.2307
Epoch [1/2], Step [201/429], Loss: 0.0637
Epoch [1/2], Step [201/429], Loss: 0.0637
Epoch [1/2], Step [301/429], Loss: 0.0534
Epoch [1/2], Step [301/429], Loss: 0.0534
Epoch [1/2], Step [401/429], Loss: 0.0045
Epoch [1/2], Step [401/429], Loss: 0.0045
Epoch 1: Average Loss: 0.0487, Accuracy: 0.9843
Epoch [2/2], Step [1/429], Loss: 0.0436
Epoch 1: Average Loss: 0.0487, Accuracy: 0.9843
Epoch [2/2], Step [1/429], Loss: 0.0436
Epoch [2/2], Step [101/429], Loss: 0.0057
Epoch [2/2], Step [101/429], Loss: 0.0057
Epoch [2/2], Step [201/429], Loss: 0.0284
Epoch [2/2], Step [201/429], Loss: 0.0284
Epoch [2/2], Step [301/429], Loss: 0.0008
Epoch [2/2], Step [301/429], Loss: 0.0008
Epoch [2/2], Step [401/429], Loss: 0.0026
Epoch [2/2], Step [401/429], Loss: 0.0026
Epoch 2: Average Loss: 0.0430, Accuracy: 0.9894
Epoch 2: Average Lo

In [6]:
torch.save(model.state_dict(), 'characters.pth')